In [2]:
# =====================================================
# FAKE NEWS DETECTION – (RoBERTa-base version) + LIAR
# =====================================================

# 1. INSTALL & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. GPU INFO
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0

print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 3. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive', force_remount=True)
    except ValueError:
        print("Drive có thể đã được mount. Bỏ qua.")

OUTPUT_DIR = "/content/drive/MyDrive/LIAR_RoBERTa_base_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET LIAR & PRE-PROCESSING
print("\n⏳ Đang tải dataset LIAR...")

# Thử tải bản sạch hơn (chengxuphd/liar2), nếu lỗi thì tải bản gốc (liar)
try:
    dataset = load_dataset("chengxuphd/liar2")
    print("✅ Đã tải 'chengxuphd/liar2'")
except Exception as e:
    print(f"⚠️ Tải liar2 thất bại ({e}), chuyển sang bản gốc 'liar'...")
    dataset = load_dataset("liar")

# Chuyển sang Pandas để xử lý linh hoạt
df_train = pd.DataFrame(dataset['train'])
df_val   = pd.DataFrame(dataset['validation'])
df_test  = pd.DataFrame(dataset['test'])
df = pd.concat([df_train, df_val, df_test], ignore_index=True)

print(f"Tổng số mẫu dữ liệu: {len(df)}")

# --- A. XỬ LÝ NHÃN (6 class -> 2 class) ---
# LIAR Labels:
# 0: false, 1: half-true, 2: mostly-true, 3: true, 4: barely-true, 5: pants-fire (Thứ tự của bản 'liar' gốc)
# Lưu ý: Bản 'liar2' có thể đã map sẵn hoặc thứ tự khác. Code này xử lý an toàn:

def map_liar_labels_safe(row):
    lbl = row['label']

    # Nếu nhãn là chuỗi (string)
    if isinstance(lbl, str):
        lbl = lbl.lower()
        if lbl in ['false', 'barely-true', 'pants-fire', 'pants-on-fire']:
            return 0 # Fake
        elif lbl in ['true', 'mostly-true', 'half-true']:
            return 1 # Real
        return 0 # Mặc định Fake nếu lỗi

    # Nếu nhãn là số (int) - Giả định theo chuẩn HuggingFace LIAR
    # Fake Group: 0 (false), 4 (barely-true), 5 (pants-fire)
    # Real Group: 1 (half-true), 2 (mostly-true), 3 (true)
    if isinstance(lbl, (int, np.integer)):
        if lbl in [0, 4, 5]: return 0
        if lbl in [1, 2, 3]: return 1
        return 0
    return 0

print("🔄 Đang chuyển đổi nhãn sang Binary (0: Fake, 1: Real)...")
df['binary_label'] = df.apply(map_liar_labels_safe, axis=1)

# --- B. FEATURE ENGINEERING CHO ROBERTA ---
# LIAR cần metadata để hiểu ngữ cảnh.
# Cấu trúc: Statement </s> Metadata
def create_liar_content(row):
    # Lấy các trường, fillna để tránh lỗi
    stmt = str(row.get('statement', '')).strip()
    speaker = str(row.get('speaker', 'Unknown'))
    party = str(row.get('party_affiliation', 'Unknown'))
    context = str(row.get('context', 'Unknown'))
    subject = str(row.get('subject', 'Unknown'))

    # Metadata string: "Obama (Democrat) | Economy | Debate"
    meta_info = f"{speaker} ({party}) | {subject} | {context}"

    # Ghép chuỗi với separator của RoBERTa
    return stmt + " </s> " + meta_info

print("🛠️ Đang tạo nội dung input (Statement + Metadata)...")
df['content'] = df.apply(create_liar_content, axis=1)

# --- C. CLEANING ---
def clean_text(s):
    if not isinstance(s, str): return ""
    s = re.sub(r'https?://\S+', ' ', s)
    # Giữ lại chữ hoa, số và ký tự đặc biệt của metadata (|, (), -)
    s = re.sub(r'[^a-zA-Z0-9\s\.\,\!\?\(\)\|\-]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 10] # Bỏ mẫu quá ngắn

print(f" → Sau xử lý: {len(df):,}")
print(f"Ví dụ: {df['content'].iloc[0]}")

# Class Weights
classes = np.unique(df['binary_label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['binary_label'])
class_weight_dict = {i: float(w) for i, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 5. SPLIT DATA
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['binary_label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['binary_label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','binary_label']].rename(columns={'binary_label':'label'}).reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','binary_label']].rename(columns={'binary_label':'label'}).reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','binary_label']].rename(columns={'binary_label':'label'}).reset_index(drop=True))
})

# 6. TOKENIZER (RoBERTa-BASE)
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("⚙️ Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 7. MODEL SETUP
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

# 8. CHECKPOINT MANAGEMENT
def get_last_checkpoint(output_dir):
    if not os.path.exists(output_dir): return None
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts: return None
    ckpts_sorted = sorted(ckpts, key=lambda x: int(x.split('-')[-1]), reverse=True)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print(f"Checkpoint: {last_checkpoint if last_checkpoint else 'Training from scratch'}")

# 9. CUSTOM TRAINER (Weighted Loss)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Đảm bảo weight đúng thứ tự index
        weights = [class_weight_dict[0], class_weight_dict[1]]
        w = torch.tensor(weights, dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=3e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 11. TRAIN
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN ROBERTA-BASE (LIAR)...")
trainer.train(resume_from_checkpoint=last_checkpoint)

# 12. EVALUATE & SAVE
print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

final_path = os.path.join(OUTPUT_DIR, "final_roberta_liar")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

# Dọn dẹp checkpoint
for f in os.listdir(OUTPUT_DIR):
    if f.startswith("checkpoint-"):
        shutil.rmtree(os.path.join(OUTPUT_DIR, f), ignore_errors=True)

print(f"\n✅ Đã lưu model tại: {final_path}")

Device: Tesla T4 | CUDA: True | VRAM: 15.8 GB
Mounted at /content/drive

⏳ Đang tải dataset LIAR...


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Checkpoint: Training from scratch

🚀 BẮT ĐẦU HUẤN LUYỆN ROBERTA-BASE (LIAR)...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.691000,0.674450,0.570122,0.572187,0.615229,0.570122
2,0.659400,0.643550,0.648084,0.648015,0.647948,0.648084
3,0.628300,0.653665,0.639373,0.643467,0.659414,0.639373



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.6589974761009216, 'eval_accuracy': 0.631258162821071, 'eval_f1': 0.6301567846541409, 'eval_precision': 0.6292550375151521, 'eval_recall': 0.631258162821071, 'eval_runtime': 4.8106, 'eval_samples_per_second': 477.484, 'eval_steps_per_second': 29.934, 'epoch': 3.0}

✅ Đã lưu model tại: /content/drive/MyDrive/LIAR_RoBERTa_base_Pro/final_roberta_liar
